In [32]:
import numpy as np
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
from icecream import ic
from dataclasses import dataclass

## n-puzzle

In [23]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [24]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def goal_test(state: np.ndarray) -> bool:
    c = 1
    for i in state:
        for j in i:
            if j != c:
                return False
            c += 1
            if c == len(state)**2:
                c = 0
    return True

def stateInList(state: np.ndarray, frontier: list[np.ndarray]) -> int:
    for i, element in enumerate(frontier):
        c = 0
        for x in range(PUZZLE_DIM):
            for y in range(PUZZLE_DIM):
                if state[x][y] == element[x][y]:
                    c += 1
        if c == PUZZLE_DIM**2:
            return i
    return -1


In [25]:
RANDOMIZE_STEPS = 100_000
initial_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    #scelgo randomicamente un azione da quelle possibili e la effettuo
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))
initial_state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[8, 2, 4],
       [1, 6, 3],
       [5, 7, 0]])

## depth-first approach

Memory consumption is high, we can't always find a solution if the number of iterations is too low

In [26]:
frontier = list()
visited_state = list()
cost_list = list()
cost = 0
state = initial_state
for step in tqdm(range(5000)):
    visited_state.append(state)
    if goal_test(state):
        break
    for a in available_actions(state):
        new_state = do_action(state, a)
        new_cost = cost + 1
        if stateInList(new_state, visited_state) == -1 and stateInList(new_state, frontier) == -1:
            frontier.append(new_state)
            cost_list.append(new_cost)
    state = frontier.pop()
    cost = cost_list.pop()
efficiency = cost / (step + 1)
ic(goal_test(state), state, efficiency, len(frontier), len(visited_state))
None

  0%|          | 0/5000 [00:00<?, ?it/s]

ic| goal_test(state): False
    state: array([[7, 6, 8],
                  [1, 0, 5],
                  [3, 2, 4]])
    efficiency: 0.9732
    len(frontier): 3878
    len(visited_state): 5000


## breadth-first approach

Memory consumption is high, we can't always find a solution if the number of iterations is too low

In [27]:
frontier = list()
visited_state = list()
cost_list = list()
cost = 0
state = initial_state
for step in tqdm(range(5000)):
    visited_state.append(state)
    if goal_test(state):
        break
    for a in available_actions(state):
        new_state = do_action(state, a)
        new_cost = cost + 1
        if stateInList(new_state, visited_state) == -1 and stateInList(new_state, frontier) == -1:
            frontier.append(new_state)
            cost_list.append(new_cost)
    state = frontier.pop(0)
    cost = cost_list.pop(0)
efficiency = cost / (step + 1)
ic(goal_test(state), state, efficiency, len(frontier), len(visited_state))
None

  0%|          | 0/5000 [00:00<?, ?it/s]

ic| goal_test(state): False
    state: array([[6, 2, 1],
                  [8, 5, 0],
                  [7, 3, 4]])
    efficiency: 0.003
    len(frontier): 2709
    len(visited_state): 5000


## A*

In [61]:
@dataclass
class Node:
    state: np.ndarray
    cost: int = 0
    quality: int = 0

def stateInCostList(state: np.ndarray, frontier: list[Node]) -> int:
    for i, element in enumerate(frontier):
        c = 0
        for x in range(PUZZLE_DIM):
            for y in range(PUZZLE_DIM):
                if state[x][y] == element.state[x][y]:
                    c += 1
        if c == PUZZLE_DIM**2:
            return i
    return -1

def solutionDistance(state: np.ndarray) -> int:
    counter = 0
    for x in range(len(state)):
        for y in range(len(state)):
            c = 1
            for i in range(len(state)):
                for j in range(len(state)):
                    if state[x][y] == c:
                        counter += abs(x - i) + abs(y - j)
                    c += 1
                    if c == len(state)**2:
                        c = 0
    return counter

In [63]:
frontier = list()
visited_state = list()
state = Node(initial_state, 0, 0)
for step in tqdm(range(2000)):
    visited_state.append(state)
    if goal_test(state.state):
        break
    for a in available_actions(state.state):
        new_state = do_action(state.state, a)
        if stateInCostList(new_state, visited_state) == -1 and stateInCostList(new_state, frontier) == -1:
            frontier.append(Node(new_state, state.quality + 1 + solutionDistance(new_state), state.quality + 1))
    frontier.sort(key=lambda i: i.cost, reverse = False)
    state = frontier.pop(0)
efficiency = state.quality / (step + 1)
ic(goal_test(state.state), state, efficiency, len(frontier), len(visited_state))
None

  0%|          | 0/2000 [00:00<?, ?it/s]

ic| goal_test(state.state): True
    state: Node(state=array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 0]]),
                cost=22,
                quality=22)
    efficiency: 0.017094017094017096
    len(frontier): 774
    len(visited_state): 1287


### sources
- prof example
- past exercises
- slides